In [1]:
import tensorflow as tf

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd #Basic library for all of our dataset operations
import numpy as np
import requests
import io
import statsmodels.tsa.api as smt
import statsmodels as sm
import tensorflow as tf
import pmdarima as pm
import warnings
import xgboost as xgb

import lightgbm as lgb
import gluonts
from math import sqrt
import warnings
warnings.filterwarnings('ignore')
# # import shap
# # warnings.filterwarnings("ignore") #We will use deprecated models of statmodels which throw a lot of warnings to use more modern ones

from utils.metrics import evaluate
from utils.plots import bar_metrics

from statsmodels.tsa.ar_model import AR
from random import random
from datetime import datetime
from fbprophet import Prophet
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from sklearn import linear_model, svm
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer
from sklearn.model_selection import cross_val_score, GridSearchCV
from math import sqrt
from xgboost import plot_importance, plot_tree
from statsmodels.tsa.arima_model import ARIMA
from gluonts.model.deepar import DeepAREstimator
from gluonts.trainer import Trainer
from gluonts.dataset.common import ListDataset
from gluonts.evaluation.backtest import make_evaluation_predictions
from itertools import islice
from pylab import rcParams
# progress bar
from tqdm import tqdm, tqdm_notebook
# from bayes_opt import BayesianOptimization


#Extra settings
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
plt.style.use('bmh')
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['text.color'] = 'k'
print(tf.__version__)

Importing plotly failed. Interactive plots will not work.


2.4.0


In [3]:
import math
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
raw_col = ['date', 'highest', 'high', 'mid', 'low', 'ave', 'ave_dif', 'volume', 'vol_dif', 'resid']
raw = pd.read_csv('FS443.csv',names=raw_col).drop(['ave_dif', 'vol_dif', 'resid'], axis=1)
for i in range(raw.shape[0]):
    raw.volume[i] = int(raw.volume[i].split(' ')[0].replace(',', ''))
raw.volume = raw.volume.values.astype('int')

ts_date = raw.copy().date
for itr in range(raw.shape[0]):
    date = raw.date[itr].split('/')
    date[0] = str(int(date[0]) + 1911)
    result = '/'.join(date)
    ts_date[itr] =  np.datetime64(datetime.strptime(result, "%Y/%m/%d").date())
    


ts = raw.copy()
ts.date = ts_date
ts['date'] = pd.to_datetime(ts['date'], errors='coerce')
ts = ts.set_index('date')
ts.index = pd.to_datetime(ts.index)
ts

,highest,high,mid,low,ave,volume
date,,,,,,
1998-12-01,195,131,77,59,84.0,327
1998-12-02,165,144,111,93,113.0,456
1998-12-03,204,160,82,65,94.0,432
1998-12-04,189,161,114,80,116.0,583
1998-12-06,181,157,90,55,96.0,1052
...,...,...,...,...,...,...
2020-11-26,201,201,137,67,134.9,97
2020-11-27,170,170,112,52,111.8,176
2020-11-28,171,171,170,152,166.7,81


In [5]:
df = ts.copy()
df['date'] = ts.index
df['hour'] = df['date'].dt.hour.astype(int)
df['dayofweek'] = df['date'].dt.dayofweek.astype(int)
df['quarter'] = df['date'].dt.quarter.astype(int)
df['month'] = df['date'].dt.month.astype(int)
df['year'] = df['date'].dt.year.astype(int)
df['dayofyear'] = df['date'].dt.dayofyear.astype(int)
df['dayofmonth'] = df['date'].dt.day.astype(int)
df['weekofyear'] = df['date'].dt.weekofyear.astype(int)
df = df.drop(['hour','date'], axis=1)

In [17]:
ts

,highest,high,mid,low,ave,volume
date,,,,,,
1998-12-01,195,131,77,59,84.0,327
1998-12-02,165,144,111,93,113.0,456
1998-12-03,204,160,82,65,94.0,432
1998-12-04,189,161,114,80,116.0,583
1998-12-06,181,157,90,55,96.0,1052
...,...,...,...,...,...,...
2020-11-26,201,201,137,67,134.9,97
2020-11-27,170,170,112,52,111.8,176
2020-11-28,171,171,170,152,166.7,81


In [57]:
training_set = ts.ave[:-14].values.reshape(-1, 1)
test_set = ts.ave[-14:].values.reshape(-1, 1)

In [58]:
training_set.shape

(6630, 1)

In [42]:
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)
dataset = tf.data.Dataset.from_tensor_slices(training_set_scaled)
dataset = dataset.window(10, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(10))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))

train_x = []
train_y = []
for x,y in dataset:
    train_x.append(x.numpy())
    train_y.append(y.numpy())

X_train, y_train = np.array(train_x), np.array(train_y)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
print(X_train.shape)

(6621, 9, 1)


In [45]:
inputs = ts[-23:].ave.values.reshape(-1,1)
inputs = sc.transform(inputs)

dataset = tf.data.Dataset.from_tensor_slices(inputs) #20 = 14 + (5-1) -> windows size -1
dataset = dataset.window(10, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(10))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
# dataset = dataset.shuffle(buffer_size=10)
test_x = []
test_y = []
for x,y in dataset:
    test_x.append(x.numpy())
    test_y.append(y.numpy())

X_test, y_test = np.array(test_x), np.array(test_y)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
print(X_test.shape)

(14, 9, 1)


In [6]:
import numpy as np
from sklearn.model_selection import train_test_split

In [72]:
sc = MinMaxScaler(feature_range = (0, 1))
trans = sc.fit_transform(ts.ave.values.reshape(-1, 1))

In [76]:
trans.ravel().shape

(6644,)

In [77]:
def window_dataset_nodate(ts,df,window_size):
    w = window_size
    
    sc = MinMaxScaler(feature_range = (0, 1))
    trans = sc.fit_transform(ts.ave.values.reshape(-1, 1)).ravel()



    dataset = tf.data.Dataset.from_tensor_slices(trans)
    dataset = dataset.window(w, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(w))
    dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
    target = []
    x_ser = []
    for x,y in dataset:
        x_ser.append(x.numpy().ravel())
        target.append(y.numpy())
        

    return np.array(x_ser), np.array(target),sc

In [17]:
# def window_dataset_nodate(ts,df,window_size):
#     w = window_size
    
#     sc = MinMaxScaler(feature_range = (0, 1))
#     training_set_scaled = sc.fit_transform(df.values)
#     sc.transform(inputs)
#     dataset = tf.data.Dataset.from_tensor_slices(df.values)
#     dataset = dataset.window(w, shift=1, drop_remainder=True)
#     dataset = dataset.flat_map(lambda window: window.batch(w))
#     dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
#     target = []
#     x_date = []
#     for x,y in dataset:


#         target.append(y.numpy()[0,4])


#     dataset = tf.data.Dataset.from_tensor_slices(ts.ave.values)
#     dataset = dataset.window(w, shift=1, drop_remainder=True)
#     dataset = dataset.flat_map(lambda window: window.batch(w))
#     dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
#     x_ser = []
#     for x,y in dataset:
#         x_ser.append(x.numpy().ravel())
        

#     return np.array(x_ser), np.array(target)

In [94]:
x_df, y_df, sc = window_dataset_nodate(ts,df,10)

In [95]:

X_train, X_test, y_train, y_test = train_test_split(
    x_df, y_df, test_size=0.002107164358819988, random_state=0)



# sc = MinMaxScaler(feature_range = (0, 1))
# X_train = sc.fit_transform(X_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# X_test = sc.transform(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [96]:
model = Sequential()
#Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))
# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
# Adding the output layer
model.add(Dense(units = 1))

# Compiling the RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
model.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
207/207 [==============================] - 14s 17ms/step - loss: 0.0311
Epoch 2/100
207/207 [==============================] - 4s 17ms/step - loss: 0.0109
Epoch 3/100
207/207 [==============================] - 4s 17ms/step - loss: 0.0098
Epoch 4/100
207/207 [==============================] - 4s 17ms/step - loss: 0.0080
Epoch 5/100
207/207 [==============================] - 4s 17ms/step - loss: 0.0067
Epoch 6/100
207/207 [==============================] - 3s 17ms/step - loss: 0.0057
Epoch 7/100
207/207 [==============================] - 3s 17ms/step - loss: 0.0055
Epoch 8/100
207/207 [==============================] - 3s 17ms/step - loss: 0.0053
Epoch 9/100
207/207 [==============================] - 4s 17ms/step - loss: 0.0053
Epoch 10/100
207/207 [==============================] - 4s 19ms/step - loss: 0.0050
Epoch 11/100
207/207 [==============================] - 4s 20ms/step - loss: 0.0050
Epoch 12/100
207/207 [==============================] - 4s 19ms/step - loss: 0.0048


In [93]:
predicted_stock_price.shape

(14, 1)

In [98]:
predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [105]:
sc.inverse_transform(y_test)

array([[149.],
       [145.],
       [195.],
       [104.],
       [136.],
       [171.],
       [174.],
       [158.],
       [114.],
       [ 96.],
       [210.],
       [152.],
       [151.],
       [ 75.]])

In [103]:
predicted_stock_price

array([[158.82239],
       [150.1323 ],
       [198.00038],
       [118.53249],
       [153.83786],
       [198.14963],
       [158.80101],
       [174.96297],
       [128.14429],
       [ 98.81711],
       [211.452  ],
       [126.92655],
       [149.32808],
       [ 87.73979]], dtype=float32)

In [106]:
mse0 = ((sc.inverse_transform(y_test) - predicted_stock_price) ** 2).mean()

print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse0), 2)))

The Root Mean Squared Error of our forecasts is 14.44


In [107]:

X_train, X_test, y_train, y_test = train_test_split(
    x_df, y_df, test_size=0.002107164358819988, random_state=1)



# sc = MinMaxScaler(feature_range = (0, 1))
# X_train = sc.fit_transform(X_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# X_test = sc.transform(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
model = Sequential()
#Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))
# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
# Adding the output layer
model.add(Dense(units = 1))

# Compiling the RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
model.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
207/207 [==============================] - 22s 21ms/step - loss: 0.0294
Epoch 2/100
207/207 [==============================] - 4s 21ms/step - loss: 0.0115
Epoch 3/100
207/207 [==============================] - 4s 20ms/step - loss: 0.0105
Epoch 4/100
207/207 [==============================] - 4s 19ms/step - loss: 0.0082
Epoch 5/100
207/207 [==============================] - 4s 21ms/step - loss: 0.0069
Epoch 6/100
207/207 [==============================] - 4s 21ms/step - loss: 0.0060
Epoch 7/100
207/207 [==============================] - 4s 20ms/step - loss: 0.0054
Epoch 8/100
207/207 [==============================] - 4s 20ms/step - loss: 0.0055
Epoch 9/100
207/207 [==============================] - 4s 21ms/step - loss: 0.0049
Epoch 10/100
207/207 [==============================] - 4s 21ms/step - loss: 0.0049: 0s 
Epoch 11/100
207/207 [==============================] - 4s 19ms/step - loss: 0.0050
Epoch 12/100
207/207 [==============================] - 4s 20ms/step - loss: 0.

207/207 [==============================] - 4s 17ms/step - loss: 0.0041
Epoch 99/100
207/207 [==============================] - 4s 17ms/step - loss: 0.0040
Epoch 100/100
207/207 [==============================] - 4s 18ms/step - loss: 0.0042


In [109]:
predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)
mse1 = ((sc.inverse_transform(y_test) - predicted_stock_price) ** 2).mean()

print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse1), 2)))

The Root Mean Squared Error of our forecasts is 27.35


In [110]:

X_train, X_test, y_train, y_test = train_test_split(
    x_df, y_df, test_size=0.002107164358819988, random_state=2)



sc = MinMaxScaler(feature_range = (0, 1))
X_train = sc.fit_transform(X_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

X_test = sc.transform(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
model = Sequential()
#Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))
# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
# Adding the output layer
model.add(Dense(units = 1))

# Compiling the RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
model.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
207/207 [==============================] - 19s 19ms/step - loss: 0.0305
Epoch 2/100
207/207 [==============================] - 4s 20ms/step - loss: 0.0117
Epoch 3/100
207/207 [==============================] - 4s 21ms/step - loss: 0.0097
Epoch 4/100
207/207 [==============================] - 4s 21ms/step - loss: 0.0077: 
Epoch 5/100
207/207 [==============================] - 4s 22ms/step - loss: 0.0071
Epoch 6/100
207/207 [==============================] - 4s 20ms/step - loss: 0.0058: 0
Epoch 7/100
207/207 [==============================] - 4s 20ms/step - loss: 0.0052
Epoch 8/100
207/207 [==============================] - 4s 20ms/step - loss: 0.0053
Epoch 9/100
207/207 [==============================] - 4s 20ms/step - loss: 0.0048
Epoch 10/100
207/207 [==============================] - 4s 21ms/step - loss: 0.0052
Epoch 11/100
207/207 [==============================] - 4s 20ms/step - loss: 0.0047
Epoch 12/100
207/207 [==============================] - 4s 21ms/step - loss: 0.

In [ ]:
predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)
mse2 = ((sc.inverse_transform(y_test) - predicted_stock_price) ** 2).mean()

print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse2), 2)))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    x_df, y_df, test_size=0.002107164358819988, random_state=3)



sc = MinMaxScaler(feature_range = (0, 1))
X_train = sc.fit_transform(X_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

X_test = sc.transform(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
model = Sequential()
#Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))
# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
# Adding the output layer
model.add(Dense(units = 1))

# Compiling the RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
model.fit(X_train, y_train, epochs = 100, batch_size = 32)

In [ ]:
predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)
mse3 = ((sc.inverse_transform(y_test) - predicted_stock_price) ** 2).mean()

print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse3), 2)))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    x_df, y_df, test_size=0.002107164358819988, random_state=4)



sc = MinMaxScaler(feature_range = (0, 1))
X_train = sc.fit_transform(X_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

X_test = sc.transform(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
model = Sequential()
#Adding the first LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))
# Adding a second LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a third LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
# Adding a fourth LSTM layer and some Dropout regularisation
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
# Adding the output layer
model.add(Dense(units = 1))

# Compiling the RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
model.fit(X_train, y_train, epochs = 100, batch_size = 32)

In [ ]:
predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)
mse4 = ((sc.inverse_transform(y_test) - predicted_stock_price) ** 2).mean()

print('The Root Mean Squared Error of our forecasts is {}'.format(round(np.sqrt(mse4), 2)))